In [ ]:
1import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
from datetime import datetime

# Extracción

1.Cargar el dataset desde un archivo Excel (Dataset_Clientes_Colombia.xlsx).


In [ ]:
df_data = pd.read_excel("/content/Dataset_Clientes_Colombia.xlsx")
df_data

,ID Compra,Nombre,Apellido,Electrodoméstico,Cantidad,Valor Unitario,Valor Total,Fecha Compra,Hora Compra
0,CL-00001,Marcia,PARRA,Nevera,5.0,1800000,9000000,2023-07-12,03:27:54
1,CL-00002,Alfonso,ROMA,Plancha,1.0,150000,150000,2023-01-16,04:34:05
2,CL-00003,NaN,DUE%AS,Horno eléctrico,3.0,500000,1500000,2022-09-21,20:18:45
3,CL-00004,Gonzalo,SEBASTIÁN,Licuadora,3.0,250000,750000,2024-05-23,15:39:00
4,CL-00005,Ambrosio,PRAT,Lavadora,5.0,1500000,7500000,2022-09-08,13:57:58
...,...,...,...,...,...,...,...,...,...
11270,CL-11271,Eusebia,BÁRCENA,Aspiradora,1.0,600000,600000,2024-04-02,21:34:06
11271,CL-11272,Trini,GALLO,Microondas,3.0,400000,1200000,2023-08-29,01:20:59
11272,CL-11273,Bautista,VILLAVERDE,Lavadora,5.0,1500000,7500000,2024-03-04,05:50:19
11273,CL-11274,Jordana,CABRERA,Cafetera,5.0,300000,1500000,2022-05-26,21:24:03


2.Identificar y contar:
•Valores nulos
•Valores vacíos (strings vacíos)
•Filas duplicadas (con y sin considerar el ID)

In [ ]:
df_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11275 entries, 0 to 11274
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   ID Compra         11275 non-null  object 
 1   Nombre            11254 non-null  object 
 2   Apellido          11255 non-null  object 
 3   Electrodoméstico  11254 non-null  object 
 4   Cantidad          11265 non-null  float64
 5   Valor Unitario    11275 non-null  int64  
 6   Valor Total       11275 non-null  int64  
 7   Fecha Compra      11275 non-null  object 
 8   Hora Compra       11275 non-null  object 
dtypes: float64(1), int64(2), object(6)
memory usage: 792.9+ KB


In [ ]:
df_data.isnull().sum() # muestra  nulos de cada columna

,0
ID Compra,0
Nombre,21
Apellido,20
Electrodoméstico,21
Cantidad,10
Valor Unitario,0
Valor Total,0
Fecha Compra,0
Hora Compra,0


In [ ]:
duplicados_total = df_data[df_data.duplicated(keep=False)] ## duplicadas considerando todo
print(duplicados_total) # No genera duplicados

Empty DataFrame
Columns: [ID Compra, Nombre, Apellido, Electrodoméstico, Cantidad, Valor Unitario, Valor Total, Fecha Compra, Hora Compra]
Index: []


In [ ]:
## duplicadas sin considerar 'ID compra'
duplicados_sin_id = df_data[df_data.drop(columns=['ID Compra']).duplicated(keep=False)]
print(duplicados_sin_id) # Genera varios duplicados que a parecer son los electrodomésticos

      ID Compra     Nombre    Apellido Electrodoméstico  Cantidad  \
0      CL-00001     Marcia       PARRA           Nevera       5.0   
1      CL-00002    Alfonso        ROMA          Plancha       1.0   
3      CL-00004    Gonzalo   SEBASTIÁN        Licuadora       3.0   
218    CL-00219       Iker        VERA         Cafetera       4.0   
795    CL-00796   Domitila     MARQUEZ  Horno eléctrico       3.0   
1496   CL-01497   Cipriano     VERDUGO         Cafetera       4.0   
1501   CL-01502      Trini       GALLO       Microondas       3.0   
2252   CL-02253     Ileana       CAMPS        Licuadora       3.0   
4373   CL-04374  Prudencio    QUINTERO       Ventilador       2.0   
4463   CL-04464       Cruz      ADADIA         Lavadora       1.0   
4836   CL-04837       Luís     CARDONA           Nevera       1.0   
5137   CL-05138    Jordana     CABRERA         Cafetera       5.0   
5753   CL-05754     Azahar    FIGUEROA         Lavadora       4.0   
5833   CL-05834       Vito    GUER

3.Verificar formatos inconsistentes en:
•Fecha Compra
•Nombre y Apellido (mayúsculas, símbolos)
•Entre otros
Transformación

In [ ]:
df_data.loc[:,'Fecha Compra'] #trae todas las fechas

,Fecha Compra
0,2023-07-12
1,2023-01-16
2,2022-09-21
3,2024-05-23
4,2022-09-08
...,...
11270,2024-04-02
11271,2023-08-29
11272,2024-03-04
11273,2022-05-26


In [ ]:
df_data['Fecha Compra'] = pd.to_datetime(df_data['Fecha Compra'], errors='coerce') ## Valido Fechas inconsistentes
fechas_invalidas = df_data[df_data['Fecha Compra'].isnull()]
print(fechas_invalidas)  ## 150 filas con fechas inconsistentes

      ID Compra      Nombre  Apellido Electrodoméstico  Cantidad  \
57     CL-00058     Alondra    ALEGRE       Microondas       3.0   
146    CL-00147       Nuria  ZORRILLA           Nevera       1.0   
227    CL-00228   Fortunata  ESCOLANO       Microondas       2.0   
344    CL-00345     Clímaco    GASCÓN           Nevera       2.0   
450    CL-00451     Antonio  SANTIAGO       Microondas       5.0   
...         ...         ...       ...              ...       ...   
10844  CL-10845    Anacleto   MONTA%A       Aspiradora       2.0   
10999  CL-11000  Evangelina     MATAS           Nevera       3.0   
11040  CL-11041    Fernanda     NU%EZ          Plancha       1.0   
11165  CL-11166    Cristina     MORÁN         Lavadora       2.0   
11254  CL-11255    Ambrosio      PRAT         Lavadora       5.0   

       Valor Unitario  Valor Total Fecha Compra Hora Compra  
57             400000      1200000          NaT    01:29:35  
146           1800000      1800000          NaT    06:22:49

In [ ]:
# Verificar nombres con símbolos no deseados
nombres_error = df_data[df_data['Nombre'].str.contains(r'[^\w\s]', na=False)]
print(nombres_error)

      ID Compra  Nombre     Apellido Electrodoméstico  Cantidad  \
77     CL-00078  Bego%a         EGEA          Plancha       3.0   
376    CL-00377   Í%igo     TRUJILLO        Licuadora       1.0   
1030   CL-01031  Bego%a        CERDÁ  Horno eléctrico       4.0   
1230   CL-01231    To%o        VIGIL       Ventilador       2.0   
1259   CL-01260    To%o        DIEGO           Nevera       3.0   
1378   CL-01379   Í%igo        AZNAR       Aspiradora       3.0   
1602   CL-01603   Í%igo      BARROSO       Aspiradora       1.0   
1927   CL-01928   Í%igo        OJEDA           Nevera       5.0   
2209   CL-02210   Í%igo     HIGUERAS        Televisor       4.0   
2246   CL-02247  Bego%a       LLORET       Aspiradora       2.0   
2249   CL-02250    To%o    ESCRIBANO       Microondas       4.0   
2473   CL-02474  Bego%a      SEVILLA       Aspiradora       2.0   
2709   CL-02710   Í%igo      ESPA%OL         Cafetera       4.0   
2968   CL-02969  Bego%a       CASALS       Ventilador       2.

In [ ]:
# Verificar apellidos en mayúsculas (TODO))
nombres_mayuscula = df_data[df_data['Nombre'].str.isupper() == True]
print(nombres_mayuscula) # Sinn casos

Empty DataFrame
Columns: [ID Compra, Nombre, Apellido, Electrodoméstico, Cantidad, Valor Unitario, Valor Total, Fecha Compra, Hora Compra]
Index: []


In [ ]:
apellidos_invalidos = df_data[df_data['Apellido'].str.contains(r'[^\w\s]', na=False)]
print(apellidos_invalidos) ## 421 casos de apellidos

      ID Compra         Nombre   Apellido Electrodoméstico  Cantidad  \
2      CL-00003            NaN     DUE%AS  Horno eléctrico       3.0   
12     CL-00013         Rufino     CA%ETE       Microondas       3.0   
45     CL-00046        Enrique  LARRA%AGA        Televisor       3.0   
52     CL-00053         Poncio      PE%AS        Televisor       4.0   
105    CL-00106      Guadalupe     DUE%AS         Lavadora       5.0   
...         ...            ...        ...              ...       ...   
11200  CL-11201          Belen    CABA%AS         Cafetera       2.0   
11228  CL-11229         Blanca     IBÁ%EZ       Ventilador       4.0   
11234  CL-11235  María manuela      PI%OL       Ventilador       1.0   
11236  CL-11237           Fito    ESPA%OL          Plancha       2.0   
11252  CL-11253           Joan     DUE%AS  Horno eléctrico       3.0   

       Valor Unitario  Valor Total Fecha Compra Hora Compra  
2              500000      1500000   2022-09-21    20:18:45  
12         

In [ ]:
# Verificar apellidos en mayúsculas (TODO))
apellidos_mayusculas = df_data[df_data['Apellido'].str.isupper() == True]
print(apellidos_mayusculas) ## Casi todos (11255 de 11275) tienen en mayuscula su apellido

      ID Compra     Nombre    Apellido Electrodoméstico  Cantidad  \
0      CL-00001     Marcia       PARRA           Nevera       5.0   
1      CL-00002    Alfonso        ROMA          Plancha       1.0   
2      CL-00003        NaN      DUE%AS  Horno eléctrico       3.0   
3      CL-00004    Gonzalo   SEBASTIÁN        Licuadora       3.0   
4      CL-00005   Ambrosio        PRAT         Lavadora       5.0   
...         ...        ...         ...              ...       ...   
11270  CL-11271    Eusebia     BÁRCENA       Aspiradora       1.0   
11271  CL-11272      Trini       GALLO       Microondas       3.0   
11272  CL-11273   Bautista  VILLAVERDE         Lavadora       5.0   
11273  CL-11274    Jordana     CABRERA         Cafetera       5.0   
11274  CL-11275  Prudencio    QUINTERO       Ventilador       2.0   

       Valor Unitario  Valor Total Fecha Compra Hora Compra  
0             1800000      9000000   2023-07-12    03:27:54  
1              150000       150000   2023-01-16

# Transformación

1.Limpiar los nombres y apellidos

In [ ]:
import re

# Definir la función para limpiar nombre
def limpiar_nombre(Nombre):
    """
    1. Elimina símbolos extraños.
    2. Convierte a formato Title Case: Primera letra en mayúscula, el resto en minúscula.
    """
    if isinstance(Nombre, str):  # Verificar que el valor sea  cadena
        Nombre = re.sub(r'[^\w\s]', '', Nombre)  # Remueve símbolos raros
        Nombre = Nombre.strip()  # Eliminar espacios blanco
        return Nombre.title()  # Convertir a formato Titulo
    else:
        return Nombre  # Si no es texto, devolver el valor original (puede ser NaN)

# Aplicar la función de limpieza a la columna Nombre
df_data['Nombre'] = df_data['Nombre'].apply(limpiar_nombre)

print(df_data['Nombre'].head())

# Validar que no existan más símbolos extraños
nombres_malos = df_data[df_data['Nombre'].str.contains(r'[^\w\s]', na=False)]
print(f"\nNombres con símbolos extraños después de la limpieza: {len(nombres_malos)}")


0      Marcia
1     Alfonso
2         NaN
3     Gonzalo
4    Ambrosio
Name: Nombre, dtype: object

Nombres con símbolos extraños después de la limpieza: 0


In [ ]:
import re

# Definir la función para limpiar apellidos
def limpiar_apellido(apellido):
    """
    1. Elimina símbolos extraños.
    2. Convierte a formato Title Case: Primera letra en mayúscula, el resto en minúscula.
    """
    if isinstance(apellido, str):  # Verificar que el valor sea  cadena
        apellido = re.sub(r'[^\w\s]', '', apellido)  # Remueve símbolos raros
        apellido = apellido.strip()  # Eliminar espacios blanco
        return apellido.title()  # Convertir a formato Titulo
    else:
        return apellido  # Si no es texto, devolver el valor original (puede ser NaN)

# Aplicar la función de limpieza a la columna "Apellido"
df_data['Apellido'] = df_data['Apellido'].apply(limpiar_apellido)

print(df_data['Apellido'].head())

# Validar que no existan más símbolos extraños
apellidos_invalidos = df_data[df_data['Apellido'].str.contains(r'[^\w\s]', na=False)]
print(f"\nApellidos con símbolos extraños tras limpieza: {len(apellidos_invalidos)}")


0        Parra
1         Roma
2        Dueas
3    Sebastián
4         Prat
Name: Apellido, dtype: object

Apellidos con símbolos extraños tras limpieza: 0


2.Normalizar fechas

In [ ]:
# Normalizar fechas detectando automáticamente el formato
df_data['Fecha Compra'] = pd.to_datetime(df_data['Fecha Compra'], format='mixed', dayfirst=True, errors='coerce')

# Opcional: mostrar cuántas fechas siguen siendo inválidas
fechas_invalidas = df_data[df_data['Fecha Compra'].isnull()]
print(f"Fechas inválidas después de normalizar: {len(fechas_invalidas)} filas")

# Formatear fechas válidas al formato 'YYYY-MM-DD' como texto
df_data['Fecha Compra'] = df_data['Fecha Compra'].dt.strftime('%Y-%m-%d')

print(df_data)


Fechas inválidas después de normalizar: 150 filas
      ID Compra     Nombre    Apellido Electrodoméstico  Cantidad  \
0      CL-00001     Marcia       Parra           Nevera       5.0   
1      CL-00002    Alfonso        Roma          Plancha       1.0   
2      CL-00003        NaN       Dueas  Horno eléctrico       3.0   
3      CL-00004    Gonzalo   Sebastián        Licuadora       3.0   
4      CL-00005   Ambrosio        Prat         Lavadora       5.0   
...         ...        ...         ...              ...       ...   
11270  CL-11271    Eusebia     Bárcena       Aspiradora       1.0   
11271  CL-11272      Trini       Gallo       Microondas       3.0   
11272  CL-11273   Bautista  Villaverde         Lavadora       5.0   
11273  CL-11274    Jordana     Cabrera         Cafetera       5.0   
11274  CL-11275  Prudencio    Quintero       Ventilador       2.0   

       Valor Unitario  Valor Total Fecha Compra Hora Compra  
0             1800000      9000000   2023-07-12    03:27:54

3.Imputar valores nulos (De forma lógica con base a contexto)

In [ ]:
df_data.isnull().sum() # muestra  nulos de cada columna

# ---- Normalizar Fecha Compra ----
df_data['Fecha Compra'] = pd.to_datetime(df_data['Fecha Compra'], errors='coerce', format='mixed', dayfirst=True)

# Imputar Fecha Compra con moda por Electrodoméstico
df_data['Fecha Compra'] = df_data.groupby('Electrodoméstico')['Fecha Compra'].transform(
    lambda x: x.fillna(x.mode()[0] if not x.mode().empty else pd.Timestamp('2023-01-01'))
)

# Crear Año y Mes de compra nuevamente
df_data['Año de compra'] = df_data['Fecha Compra'].dt.year
df_data['Mes de compra'] = df_data['Fecha Compra'].dt.month
df_data['Fecha Compra'] = df_data['Fecha Compra'].dt.strftime('%Y-%m-%d')

# ---- Imputar Nombre, Apellido, Electrodoméstico ----
df_data['Nombre'] = df_data['Nombre'].fillna('Desconocido')
df_data['Apellido'] = df_data['Apellido'].fillna('Desconocido')
df_data['Electrodoméstico'] = df_data['Electrodoméstico'].fillna('Desconocido')

# ---- Imputar Cantidad con media por Electrodoméstico ----
df_data['Cantidad'] = df_data.groupby('Electrodoméstico')['Cantidad'].transform(
    lambda x: x.fillna(round(x.mean())) if not x.mean() is pd.NA else 1
)


# Convertir año, mes y hora a enteros
df_data['Año de compra'] = df_data['Año de compra'].astype(int)
df_data['Mes de compra'] = df_data['Mes de compra'].astype(int)


4.Eliminar duplicados reales

In [ ]:
# Eliminar duplicados reales ignorando 'ID Compra'
df_data = df_data.drop_duplicates(subset=df_data.columns.difference(['ID Compra']), keep='first')

# Mostrar cuántos se eliminaron
print(f"Filas antes: {df_data.shape[0]}, después: {df_data.shape[0]}")


Filas antes: 11254, después: 11254


5.Crear nuevas columnas:
•Año de compra
•Mes de compra
•Hora de compra (Solo hora)
•Valor medio por unidad (valor total / cantidad)

In [ ]:


# Normalizar fechas (por si acaso) y horas
df_data['Fecha Compra'] = pd.to_datetime(df_data['Fecha Compra'], format='mixed', dayfirst=True, errors='coerce')
df_data['Hora Compra'] = pd.to_datetime(df_data['Hora Compra'], format='mixed', errors='coerce').dt.time

# Imputar fechas nulas con la moda por producto
df_data['Fecha Compra'] = df_data.groupby('Electrodoméstico')['Fecha Compra'].transform(
    lambda x: x.fillna(x.mode()[0] if not x.mode().empty else pd.Timestamp('2023-01-01'))
)

# Crear nuevas columnas
df_data['Año de compra'] = df_data['Fecha Compra'].dt.year
df_data['Mes de compra'] = df_data['Fecha Compra'].dt.month
df_data['Hora de compra'] = pd.to_datetime(df_data['Hora Compra'], format='%H:%M:%S', errors='coerce').dt.hour

# Calcular valor medio por unidad
df_data['Valor medio por unidad'] = df_data['Valor Total'] / df_data['Cantidad']

# Opcional: Redondear el valor medio
df_data['Valor medio por unidad'] = df_data['Valor medio por unidad'].round(2)

# Mostrar columnas nuevas
print(df_data[['Fecha Compra', 'Hora Compra', 'Año de compra', 'Mes de compra', 'Hora de compra', 'Valor medio por unidad']].head())


  Fecha Compra Hora Compra  Año de compra  Mes de compra  Hora de compra  \
0   2023-07-12    03:27:54           2023              7               3   
1   2023-01-16    04:34:05           2023              1               4   
2   2022-09-21    20:18:45           2022              9              20   
3   2024-05-23    15:39:00           2024              5              15   
4   2022-09-08    13:57:58           2022              9              13   

   Valor medio por unidad  
0               1800000.0  
1                150000.0  
2                500000.0  
3                250000.0  
4               1500000.0  


6. Clasificar los electrodomésticos por categoría;
•Cocina, Confort, etc. (al gusto)

In [ ]:
# Diccionario de clasificación por categoría
categorias = {
    'Nevera': 'Cocina',
    'Horno eléctrico': 'Cocina',
    'Licuadora': 'Cocina',
    'Microondas': 'Cocina',
    'Cafetera': 'Cocina',
    'Plancha': 'Hogar',
    'Lavadora': 'Hogar',
    'Aspiradora': 'Hogar',
    'Ventilador': 'Confort',
    'Televisor': 'Entretenimiento'
}

# Crear nueva columna con la categoría
df_data['Categoría'] = df_data['Electrodoméstico'].map(categorias)
# Mostrar un resumen con electrodoméstico y su categoría
df_data[['Electrodoméstico', 'Categoría']].head(10)

<ipython-input-98-912665cafff0>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_data['Categoría'] = df_data['Electrodoméstico'].map(categorias)


,Electrodoméstico,Categoría
0,Nevera,Cocina
1,Plancha,Hogar
2,Horno eléctrico,Cocina
3,Licuadora,Cocina
4,Lavadora,Hogar
5,Nevera,Cocina
6,Microondas,Cocina
7,Nevera,Cocina
8,Ventilador,Confort
9,Nevera,Cocina


7. Generar un Excel nuevo con todas las transformaciones realizadas

In [ ]:
df_data.to_excel('Dataset_Clientes_Colombia_transformado.xlsx', index=False)
df_data

,ID Compra,Nombre,Apellido,Electrodoméstico,Cantidad,Valor Unitario,Valor Total,Fecha Compra,Hora Compra,Año de compra,Mes de compra,Hora de compra,Valor medio por unidad,Categoría
0,CL-00001,Marcia,Parra,Nevera,5.0,1800000,9000000,2023-07-12,2025-03-21 03:27:54,2023,7,3,1800000.0,Cocina
1,CL-00002,Alfonso,Roma,Plancha,1.0,150000,150000,2023-01-16,2025-03-21 04:34:05,2023,1,4,150000.0,Hogar
2,CL-00003,Desconocido,Dueas,Horno eléctrico,3.0,500000,1500000,2022-09-21,2025-03-21 20:18:45,2022,9,20,500000.0,Cocina
3,CL-00004,Gonzalo,Sebastián,Licuadora,3.0,250000,750000,2024-05-23,2025-03-21 15:39:00,2024,5,15,250000.0,Cocina
4,CL-00005,Ambrosio,Prat,Lavadora,5.0,1500000,7500000,2022-09-08,2025-03-21 13:57:58,2022,9,13,1500000.0,Hogar
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11249,CL-11250,Edgardo,Galván,Aspiradora,3.0,600000,1800000,2022-09-11,2025-03-21 20:57:28,2022,9,20,600000.0,Hogar
11252,CL-11253,Joan,Dueas,Horno eléctrico,3.0,500000,1500000,2022-09-21,2025-03-21 20:18:45,2022,9,20,500000.0,Cocina
11254,CL-11255,Ambrosio,Prat,Lavadora,5.0,1500000,7500000,2022-03-30,2025-03-21 13:57:58,2022,3,13,1500000.0,Hogar
11258,CL-11259,Cleto,Galván,Ventilador,3.0,200000,600000,2024-06-08,2025-03-21 09:14:41,2024,6,9,200000.0,Confort


# Encontrar hallazgos con Pandas:

1.¿Cuál es el electrodoméstico más vendido por cantidad?
2.¿Cuál generó más ingresos (Valor Total)?
3.¿Cuál es el ticket promedio por compra?
4.¿Cuántas compras se realizaron por año?
5.¿Y por mes?
6.¿Cuál es la hora del día con más compras?
7.¿Qué cliente compró más unidades?
8.¿Cuántas compras tienen un valor total mayor a $5.000.000?
9.¿Cuál es la media y mediana del Valor Unitario?
10.¿Cuántas compras no tienen registrado el nombre del cliente?
11.¿Cuál es el producto más común entre las compras duplicadas?
12.¿Hay alguna relación entre la cantidad comprada y el valor total?
13.¿Cuántas categorías de electrodomésticos hay?
14.¿Qué categoría vende más?
15.¿Hay estacionalidad en las compras? ¿En qué meses se compra más?
16.¿Cuál es la proporción entre compras de lavadoras y neveras?
17.¿Cuántas compras hay de productos de cocina vs. climatización?
18.¿Cuántos productos se vendieron en total?
19.¿Cuál es el promedio de productos por compra?
20.¿Qué porcentaje de las compras tiene más de una unidad?

In [ ]:
# 1. Electrodoméstico más vendido por cantidad
mas_vendido = df_data.groupby('Electrodoméstico')['Cantidad'].sum().idxmax()

# 2. Electrodoméstico que generó más ingresos (Valor Total)
mas_ingresos = df_data.groupby('Electrodoméstico')['Valor Total'].sum().idxmax()

# 3. Ticket promedio por compra
ticket_promedio = df_data['Valor Total'].mean()

# 4. Compras por año
df_data['Fecha Compra'] = pd.to_datetime(df_data['Fecha Compra'])
compras_por_anio = df_data['Fecha Compra'].dt.year.value_counts().sort_index()

# 5. Compras por mes
compras_por_mes = df_data['Fecha Compra'].dt.month.value_counts().sort_index()

# 6. Hora del día con más compras
df_data['Hora Compra'] = pd.to_datetime(df_data['Hora Compra'].astype(str))
hora_con_mas_compras = df_data['Hora Compra'].dt.hour.value_counts().idxmax()

# 7. Cliente que compró más unidades
cliente_top_unidades = df_data.groupby('Nombre')['Cantidad'].sum().idxmax()

# 8. Compras con valor total > 5.000.000
compras_mayores_5m = df_data[df_data['Valor Total'] > 5000000].shape[0]

# 9. Media y mediana del Valor Unitario
media_valor_unitario = df_data['Valor Unitario'].mean()
mediana_valor_unitario = df_data['Valor Unitario'].median()

# 10. Compras sin nombre del cliente
compras_sin_nombre = df_data['Nombre'].isna().sum()

# 11. Producto más común en compras duplicadas
# 1. Electrodoméstico más vendido por cantidad
mas_vendido = df_data.groupby('Electrodoméstico')['Cantidad'].sum().idxmax()

# 2. Electrodoméstico que generó más ingresos (Valor Total)
mas_ingresos = df_data.groupby('Electrodoméstico')['Valor Total'].sum().idxmax()

# 3. Ticket promedio por compra
ticket_promedio = df_data['Valor Total'].mean()

# 4. Compras por año
df_data['Fecha Compra'] = pd.to_datetime(df_data['Fecha Compra'])
compras_por_anio = df_data['Fecha Compra'].dt.year.value_counts().sort_index()

# 5. Compras por mes
compras_por_mes = df_data['Fecha Compra'].dt.month.value_counts().sort_index()

# 6. Hora del día con más compras
df_data['Hora Compra'] = pd.to_datetime(df_data['Hora Compra'].astype(str))
hora_con_mas_compras = df_data['Hora Compra'].dt.hour.value_counts().idxmax()

# 7. Cliente que compró más unidades
cliente_top_unidades = df_data.groupby('Nombre')['Cantidad'].sum().idxmax()

# 8. Compras con valor total > 5.000.000
compras_mayores_5m = df_data[df_data['Valor Total'] > 5000000].shape[0]

# 9. Media y mediana del Valor Unitario
media_valor_unitario = df_data['Valor Unitario'].mean()
mediana_valor_unitario = df_data['Valor Unitario'].median()

# 10. Compras sin nombre del cliente
compras_sin_nombre = df_data['Nombre'].isna().sum()

# 11. Producto más común en compras duplicadas
duplicados = df_data[df_data.duplicated(subset=['ID Compra'], keep=False)]
if not duplicados.empty:
    modo = duplicados['Electrodoméstico'].mode()
    if not modo.empty:
        producto_mas_comun_duplicados = modo[0]
    else:
        producto_mas_comun_duplicados = "No hay modo disponible"
else:
    producto_mas_comun_duplicados = "No hay compras duplicadas"

# 12. Relación entre cantidad y valor total (correlación)
correlacion = df_data['Cantidad'].corr(df_data['Valor Total'])

# 13. Cantidad de categorías
categorias_unicas = df_data['Categoría'].nunique()

# 14. Categoría que más vende (por cantidad)
categoria_mas_vendida = df_data.groupby('Categoría')['Cantidad'].sum().idxmax()

# 15. Estacionalidad: compras por mes total
compras_estacionalidad = df_data['Fecha Compra'].dt.month.value_counts().sort_index()

# 16. Proporción lavadoras vs neveras
lavadoras = df_data[df_data['Electrodoméstico'] == 'Lavadora'].shape[0]
neveras = df_data[df_data['Electrodoméstico'] == 'Nevera'].shape[0]
proporcion_ln = lavadoras / neveras if neveras != 0 else 'No hay neveras'

# 17. Compras de cocina vs climatización
cocina = df_data[df_data['Categoría'] == 'Cocina'].shape[0]
clima = df_data[df_data['Categoría'] == 'Climatización'].shape[0]

# 18. Total de productos vendidos
total_productos = df_data['Cantidad'].sum()

# 19. Promedio de productos por compra
promedio_por_compra = df_data['Cantidad'].mean()

# 20. % de compras con más de una unidad
porcentaje_mas_de_una = (df_data[df_data['Cantidad'] > 1].shape[0] / df_data.shape[0]) * 100


# 12. Relación entre cantidad y valor total (correlación)
correlacion = df_data['Cantidad'].corr(df_data['Valor Total'])

# 13. Cantidad de categorías
categorias_unicas = df_data['Categoría'].nunique()

# 14. Categoría que más vende (por cantidad)
categoria_mas_vendida = df_data.groupby('Categoría')['Cantidad'].sum().idxmax()

# 15. Estacionalidad: compras por mes total
compras_estacionalidad = df_data['Fecha Compra'].dt.month.value_counts().sort_index()

# 16. Proporción lavadoras vs neveras
lavadoras = df_data[df_data['Electrodoméstico'] == 'Lavadora'].shape[0]
neveras = df_data[df_data['Electrodoméstico'] == 'Nevera'].shape[0]
proporcion_ln = lavadoras / neveras if neveras != 0 else 'No hay neveras'

# 17. Compras de cocina vs climatización
cocina = df_data[df_data['Categoría'] == 'Cocina'].shape[0]
clima = df_data[df_data['Categoría'] == 'Climatización'].shape[0]

# 18. Total de productos vendidos
total_productos = df_data['Cantidad'].sum()

# 19. Promedio de productos por compra
promedio_por_compra = df_data['Cantidad'].mean()

# 20. % de compras con más de una unidad
porcentaje_mas_de_una = (df_data[df_data['Cantidad'] > 1].shape[0] / df_data.shape[0]) * 100


<ipython-input-100-55cca4a48cc5>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_data['Fecha Compra'] = pd.to_datetime(df_data['Fecha Compra'])
<ipython-input-100-55cca4a48cc5>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_data['Hora Compra'] = pd.to_datetime(df_data['Hora Compra'].astype(str))
<ipython-input-100-55cca4a48cc5>:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in

In [ ]:
# imprimir resultados
print(f"1. Electrodoméstico más vendido: {mas_vendido}")
print(f"2. Electrodoméstico con más ingresos: {mas_ingresos}")
print(f"3. Ticket promedio: ${ticket_promedio:,.2f}")
print(f"4. Compras por año:\n{compras_por_anio}\n")
print(f"5. Compras por mes:\n{compras_por_mes}\n")
print(f"6. Hora con más compras: {hora_con_mas_compras}:00 hrs")
print(f"7. Cliente con más unidades compradas: {cliente_top_unidades}")
print(f"8. Compras mayores a $5.000.000: {compras_mayores_5m}")
print(f"9. Media Valor Unitario: ${media_valor_unitario:,.2f}, Mediana: ${mediana_valor_unitario:,.2f}")
print(f"10. Compras sin nombre de cliente: {compras_sin_nombre}")

if not duplicados.empty:
    print(f"11. Producto más común en compras duplicadas: {producto_mas_comun_duplicados}")
else:
    print("11. No hay compras duplicadas")

print(f"12. Correlación cantidad - valor total: {correlacion:.4f}")

if 'Categoría' in df_data.columns:
    print(f"13. Cantidad de categorías: {categorias_unicas}")
    print(f"14. Categoría más vendida: {categoria_mas_vendida}")
    print(f"17. Compras Cocina: {cocina}, Climatización: {clima}")
else:
    print("13. Columna 'Categoría' no encontrada")
    print("14. Columna 'Categoría' no encontrada")
    print("17. Columna 'Categoría' no encontrada")

print(f"15. Compras totales por mes:\n{compras_estacionalidad}\n")
print(f"16. Proporción Lavadoras/Neveras: {proporcion_ln}")
print(f"18. Total productos vendidos: {total_productos}")
print(f"19. Promedio productos por compra: {promedio_por_compra:.2f}")
print(f"20. % de compras con más de una unidad: {porcentaje_mas_de_una:.2f}%")


1. Electrodoméstico más vendido: Plancha
2. Electrodoméstico con más ingresos: Nevera
3. Ticket promedio: $2,049,893.37
4. Compras por año:
Fecha Compra
2022    3777
2023    3726
2024    3751
Name: count, dtype: int64

5. Compras por mes:
Fecha Compra
1     1031
2      834
3      930
4      917
5      924
6      911
7      923
8     1039
9      916
10     934
11     954
12     941
Name: count, dtype: int64

6. Hora con más compras: 12:00 hrs
7. Cliente con más unidades compradas: Guadalupe
8. Compras mayores a $5.000.000: 1325
9. Media Valor Unitario: $687,373.38, Mediana: $400,000.00
10. Compras sin nombre de cliente: 0
11. No hay compras duplicadas
12. Correlación cantidad - valor total: 0.4649
13. Cantidad de categorías: 4
14. Categoría más vendida: Cocina
17. Compras Cocina: 5638, Climatización: 0
15. Compras totales por mes:
Fecha Compra
1     1031
2      834
3      930
4      917
5      924
6      911
7      923
8     1039
9      916
10     934
11     954
12     941
Name: count, 

Preguntas de Negocio :

In [ ]:
# 1. Productos más vendidos en temporadas altas (ej: diciembre)
temporada_alta = df_data[df_data['Fecha Compra'].dt.month.isin([11, 12])]
productos_top_temporada = temporada_alta.groupby('Electrodoméstico')['Cantidad'].sum().sort_values(ascending=False).head(5)
print("1. Productos para promocionar en temporadas altas:\n", productos_top_temporada, "\n")

# 2. Clientes frecuentes o con tickets altos
clientes_frecuentes = df_data['Nombre'].value_counts().head(5)
clientes_tickets_altos = df_data.groupby('Nombre')['Valor Total'].sum().sort_values(ascending=False).head(5)
print("2. Clientes frecuentes:\n", clientes_frecuentes, "\n")
print("   Clientes con mayores tickets:\n", clientes_tickets_altos, "\n")

# 3. Productos con pocas ventas (evaluar si vale la pena mantenerlos)
ventas_por_producto = df_data.groupby('Electrodoméstico')['Cantidad'].sum().sort_values()
productos_baja_rotacion = ventas_por_producto.head(5)
print("3. Productos con bajas ventas (evaluar portafolio):\n", productos_baja_rotacion, "\n")

# 4. Meses con más compras (refuerzo logístico)
compras_mes = df_data['Fecha Compra'].dt.month.value_counts().sort_index()
print("4. Meses con más compras:\n", compras_mes, "\n")

# 5. Productos frecuentes en compras duplicadas (posibles errores)
duplicados = df_data[df_data.duplicated(subset=['ID Compra'], keep=False)]
productos_errores = duplicados['Electrodoméstico'].value_counts().head(3)
print("5. Productos comunes en duplicados (revisar errores de registro):\n", productos_errores, "\n")

# 6. Información útil para proyecciones de stock
ventas_mensuales = df_data.groupby([df_data['Fecha Compra'].dt.to_period('M'), 'Electrodoméstico'])['Cantidad'].sum().unstack().fillna(0)
print("6. Ventas mensuales por producto (útil para stock):\n", ventas_mensuales.tail(), "\n")

# 7. Productos comprados juntos (potenciales combos)
combos = df_data.groupby('ID Compra')['Electrodoméstico'].apply(list)
from collections import Counter
combo_counts = Counter()
for combo in combos:
    combo_counts.update(set(combo))
productos_comunes = combo_counts.most_common(5)
print("7. Productos frecuentes en combos:\n", productos_comunes, "\n")

# 8. Categorías cruzadas por cliente (ventas cruzadas)
if 'Categoría' in df_data.columns:
    categorias_por_cliente = df_data.groupby('Nombre')['Categoría'].nunique()
    potencial_cruzada = categorias_por_cliente[categorias_por_cliente > 1].count()
    print(f"8. Clientes con compras en múltiples categorías (potencial venta cruzada): {potencial_cruzada}\n")
else:
    print("8. Columna 'Categoría' no disponible\n")

# 9. Franjas horarias con más compras (para promociones digitales)
compras_por_hora = df_data['Hora Compra'].dt.hour.value_counts().sort_index()
print("9. Compras por hora (usar para promociones digitales):\n", compras_por_hora, "\n")

# 10. Compras inusuales (valor alto)
limite_alerta = df_data['Valor Total'].mean() + 3 * df_data['Valor Total'].std()
compras_inusuales = df_data[df_data['Valor Total'] > limite_alerta]
print(f"10. Compras inusuales detectadas (> ${limite_alerta:,.2f}): {compras_inusuales.shape[0]} registros\n")


1. Productos para promocionar en temporadas altas:
 Electrodoméstico
Plancha            663.0
Televisor          604.0
Horno eléctrico    584.0
Ventilador         576.0
Microondas         564.0
Name: Cantidad, dtype: float64 

2. Clientes frecuentes:
 Nombre
Guadalupe    35
José         32
Nazaret      29
Amor         27
Loreto       26
Name: count, dtype: int64 

   Clientes con mayores tickets:
 Nombre
José         86000000
Guadalupe    84900000
Amor         65200000
Teófila      60650000
Valero       59150000
Name: Valor Total, dtype: int64 

3. Productos con bajas ventas (evaluar portafolio):
 Electrodoméstico
Desconocido          67.0
Horno eléctrico    3183.0
Ventilador         3241.0
Televisor          3272.0
Lavadora           3340.0
Name: Cantidad, dtype: float64 

4. Meses con más compras:
 Fecha Compra
1     1031
2      834
3      930
4      917
5      924
6      911
7      923
8     1039
9      916
10     934
11     954
12     941
Name: count, dtype: int64 

5. Productos co